In [1]:
# 단어 단위 RNN

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
sentence = "Repeat is the best medicine for memory".split()

In [4]:
vocab = list(set(sentence))
print(vocab)

['for', 'medicine', 'Repeat', 'best', 'is', 'the', 'memory']


In [5]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}
word2index['<unk>']=0

In [6]:
print(word2index['memory'])

7


In [7]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'for', 2: 'medicine', 3: 'Repeat', 4: 'best', 5: 'is', 6: 'the', 7: 'memory', 0: '<unk>'}


In [8]:
print(index2word[7])

memory


In [9]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] 
    input_seq, label_seq = encoded[:-1], encoded[1:] 
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) 
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) 
    return input_seq, label_seq

In [10]:
X, Y = build_data(sentence, word2index)

In [11]:
print(X)
print(Y)

tensor([[3, 5, 6, 4, 2, 1]])
tensor([[5, 6, 4, 2, 1, 7]])


In [12]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, 
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, 
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) 

    def forward(self, x):
        output = self.embedding_layer(x)
        output, hidden = self.rnn_layer(output)
        output = self.linear(output)
        return output.view(-1, output.size(2))

In [13]:
vocab_size = len(word2index) 
input_size = 5 
hidden_size = 20 

In [14]:
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.Adam(params=model.parameters())

In [15]:
output = model(X)
print(output)

tensor([[ 0.2397,  0.1016,  0.3252, -0.0744, -0.3482,  0.1435,  0.0635, -0.2310],
        [ 0.2569, -0.0387,  0.3778, -0.0836, -0.0612, -0.0209,  0.2912, -0.2449],
        [ 0.2338,  0.1109,  0.6446,  0.1820,  0.0110,  0.1091,  0.2335, -0.6025],
        [ 0.1564,  0.2319,  0.2272, -0.3523, -0.2584, -0.1457,  0.2726,  0.0156],
        [ 0.1524, -0.0644,  0.3722, -0.1274,  0.0704, -0.0430,  0.3356, -0.2255],
        [ 0.1548,  0.2697,  0.4518, -0.0185, -0.1119,  0.0788,  0.1944, -0.3947]],
       grad_fn=<ViewBackward0>)


In [16]:
print(output.shape)

torch.Size([6, 8])


In [17]:
decode = lambda y: [index2word.get(x) for x in y]

In [21]:
for step in range(201):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()

    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1310 
Repeat medicine medicine medicine the medicine medicine

[41/201] 1.5607 
Repeat is the best medicine for for

[81/201] 0.9079 
Repeat is the best medicine for memory

[121/201] 0.4166 
Repeat is the best medicine for memory

[161/201] 0.1994 
Repeat is the best medicine for memory

[201/201] 0.1158 
Repeat is the best medicine for memory

